In [12]:
# %matplotlib
%load_ext autoreload
%autoreload 2
from functions4 import *
from declarations2 import *
from classes4 import *
import numpy as np
import SkymapPlot.SkyMap
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
#from matplotlib import rcParams
#rcParams.update({'figure.autolayout': True})
#import plotly.plotly as py
SAVE_DIRECTORY = "/net/scratch_icecube4/user/kalaczynski/Plots/"
from astropy.io import fits

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Running the main code

In [13]:
# DETECTOR="0001"  # "IC86_12_13_14" ,"IC86_11", "IC79" "IC59"   | Binary System
#                                                                    mu  #src
%run createSmearedSkymap_multipleyears.py "multi_mu_1" 0 1 "VarSig" [30] [5] detector="1111" gamma=2.13 catalog="NVSS" useCatalogPos=5 useE=False useAlm=False UseGalPlane=False 
#Theta=[130.489164] Phi=[299.259131]
#extended_source=2.
#useCatalogPos=10
#useweakestsource=10
#useDiffBG=True


DETECTOR Configuration:
['IC86_12_13_14', 'IC86_11', 'IC79', 'IC59']
/net/scratch_icecube4/user/glauch/DATA/
For Detector Configuration: IC86_12_13_14 the total uptime is 91563185.0 s
Read MC: /net/scratch_icecube4/user/glauch/DATA/IC86_2012_13_14_11029_11069_11070_corrected_ow.npy
/net/scratch_icecube4/user/glauch/DATA/
For Detector Configuration: IC86_11 the total uptime is 29556432.0 s
Read MC: /net/scratch_icecube4/user/glauch/DATA/IC86_2011_10602_11077_11191_corrected_ow.npy
/net/scratch_icecube4/user/glauch/DATA/
For Detector Configuration: IC79 the total uptime is 26784000.0 s
Read MC: /net/scratch_icecube4/user/glauch/DATA/IC79_6308_6850_corrected_ow.npy
/net/scratch_icecube4/user/glauch/DATA/
For Detector Configuration: IC59 the total uptime is 30079123.2 s
Read MC: /net/scratch_icecube4/user/glauch/DATA/IC59_6471_6515_corrected_ow.npy
nside as usual: 512
Number of Events per Chosen Sample:
[217033, 71103, 36851, 21442]
Reading Source Catalog
/net/scratch_icecube4/user/glauch/

# FITS FILES

In [30]:
#FITS files:
filename="COM_PCCS_030_R2.04"
path="/net/scratch_icecube4/user/kalaczynski/Catalogs/COM_PCCS-Catalogs_vPR2/"+str(filename)+".fits"
hdulist = fits.open(path)
#hdulist
#fitsMap=H.read_map(path)
#test=H.mrdfits(path)
#print test[1]

In [4]:
print hdulist[1].data

In [18]:
H.mollview(test[1], coord=['G','E'], title='Histogram equalized Ecliptic', unit='mK', norm='hist', min=-1,max=1, xsize=2000)
H.graticule()
plt.show()

In [6]:
#print len(catalog_map)
#print len(fitsMap)
#print len(mergeGPU.map_delta)
#print len(fitsMap)/len(catalog_map)
#fitsAlm=[]
#print path
#fitsAlm=H.read_alm(path)
#print len(fitsAlm)

In [7]:
H.gnomview(fitsMap, rot=[0,0.3], title='GnomView', unit='mK', format='%.2g')
plt.show()

In [17]:
H.mollview(mergeGPU.map_delta, rot=[0,0.3], title='GnomView', unit='mK', format='%.2g')
plt.show()

# Plotting skymap:

In [43]:
nFigure=5
Smoothing=True
Galactic=False
Catalog=False
NewMap=False
logz=True
plot=newfig(5., 3.)
Norm=None
if logz: Norm='log'
    
    
if NewMap==True:
    usedMap=newMap
else:
    if Catalog==False:
        usedMap=mergeGPU.map_delta
    else:
        usedMap=catalog_map

#usedMap=fitsMap       
#usedMap=modif    
#usedMap=map_delta2

if Smoothing:
    smoothed_map = H.smoothing(usedMap, sigma=np.radians(0.5))
else:
    smoothed_map=usedMap
if min(smoothed_map)<=0:
    raiser=abs(min(smoothed_map))*2
else:
    raiser = 0.
    
for i in range(len(smoothed_map)):
    smoothed_map[i] = float(smoothed_map[i])
    if logz:
        smoothed_map[i]=smoothed_map[i]+raiser
       

colormap=plt.get_cmap("Blues")
#colormap.set_under("white")
colormap.set_bad("white")
#col2="0."
#col="0."
res=800

smoothed_map= np.ma.masked_array(smoothed_map)
#smoothed_map.mask = np.log10(smoothed_map) < 18.1  #####BE careful..Value hardcoded
smoothed_map.mask = smoothed_map == 0

if Catalog:
    plotTitle=r"$\log_{10}(\frac{\phi}{\phi_{\mathrm{Total}}} \cdot A_{\mathrm{eff}})$"
    #plotTitle=r"$\log_{10}(\frac{\mathrm{Flux}}{\mathrm{Total Flux}} \cdot A_{\mathrm{eff}})$"
else:
    plotTitle=r"$\log_{10}(N_{\mathsf{Events}})$"

if Galactic==False:
    skymap=SkymapPlot.SkyMap.SkyMap(fig=plot)
    #smoothed_map=usedMap
    skymap.plot_map(np.log10(smoothed_map),
                    cmap=colormap, npix=res, 
                    colorbar=dict(title=plotTitle))
    if (Catalog!=True) and len(mergeGPU.phi_sou)>0:
        skymap.plot_circle(mergeGPU.phi_sou,mergeGPU.theta_sou)
    #skymap.fig.savefig(SAVE_DIRECTORY+"BGD_smoothed_equatorial.png", dpi=300, bbox_inches="tight")
else:
    skymap=SkymapPlot.SkyMap.SkyMapGalactic()
    skymap.add_plane()
    #smoothed_map=usedMap
    skymap.plot_map(np.log10(smoothed_map),
                    cmap=colormap, npix=res, 
                    colorbar=dict(title=plotTitle))
    if (Catalog!=True) and len(mergeGPU.phi_sou)>0:
        skymap.plot_circle(mergeGPU.phi_sou,mergeGPU.theta_sou)
    #skymap.fig.savefig(SAVE_DIRECTORY+"BGD_smoothed_galactic.png", dpi=300, bbox_inches="tight")
plt.show()

Sigma is 30.000000 arcmin (0.008727 rad) 
-> fwhm is 70.644601 arcmin
Sigma is 0.000000 arcmin (0.000000 rad) 
-> fwhm is 0.000000 arcmin


# Getting alms from the files:

In [28]:
l_max=1000
BGD_DIR="/net/scratch_icecube4/user/kalaczynski/Analysis_stuff/Normed/With_plane/VarSig_lmax="+str(l_max)+"/0src/Detector=1111"  

In [ ]:
# BGD

size=int((1.+(l_max))*((2.+(l_max))/2.)) #size of the alm array
no_files=1000 #max is 1000 so far
temp1=[]
temp2=[]
Alm_BGD=[]
Alm_BGD_avg=[0]*size
  
#BGD files loop:
for i in range(no_files):
    temp1=np.loadtxt(BGD_DIR+"/almAllAbs_"+str(i)+".txt")
    temp2=np.loadtxt(BGD_DIR+"/almAllPhase_"+str(i)+".txt")
    Alm_BGD.append(temp1*np.cos(temp2)+temp1*np.sin(temp2)*1j)
    Alm_BGD_avg=Alm_BGD_avg+((temp1*np.cos(temp2)+temp1*np.sin(temp2)*1j))/no_files

# save BGD_avg as files:
np.savetxt(BGD_DIR+"/Alm_BGD_avg_re.txt", np.real(Alm_BGD_avg))
np.savetxt(BGD_DIR+"/Alm_BGD_avg_im.txt", np.imag(Alm_BGD_avg))

In [46]:
#single readout:
temp1=np.loadtxt(BGD_DIR+"/almAllAbs_0.txt")
temp2=np.loadtxt(BGD_DIR+"/almAllPhase_0.txt")
Alm_BGD=temp1*np.cos(temp2)+temp1*np.sin(temp2)*1j

In [23]:
# save catalog alms:
Alm_cat=H.anafast(catalog_map, lmax=l_max, alm="True")[1]
np.savetxt(BGD_DIR+"/CatAlm_re.txt", np.real(Alm_cat))
np.savetxt(BGD_DIR+"/CatAlm_im.txt", np.imag(Alm_cat))

In [18]:
# read catalog files:
re_part=np.loadtxt(BGD_DIR+"/CatAlm_re.txt")
im_part=np.loadtxt(BGD_DIR+"/CatAlm_im.txt")
Alm_cat=re_part+im_part*1j

In [19]:
# read BGD_avg files:
re_part=np.loadtxt(BGD_DIR+"/Alm_BGD_avg_re.txt")
im_part=np.loadtxt(BGD_DIR+"/Alm_BGD_avg_im.txt")
Alm_BGD_avg=re_part+im_part*1j

In [20]:
#read signal files:
no_srcs=5
SIG_DIR="/net/scratch_icecube4/user/kalaczynski/Analysis_stuff/Normed/No_plane/VarSig_lmax="+str(l_max)+"/"+str(no_srcs)+"src/Detector=1111"
Alm_VarSig=[]

abs_sig=np.loadtxt(SIG_DIR+"/almAllAbs_0.txt")
phase_sig=np.loadtxt(SIG_DIR+"/almAllPhase_0.txt")

Alm_VarSig=abs_sig*np.cos(phase_sig)+abs_sig*np.sin(phase_sig)*1j


# Alm plotting

In [6]:
plt.xlabel("Im($a^{l}_{m}$)$\cdot 10^{6}$", fontsize=34)
plt.ylabel("Re($a^{l}_{m}$)$\cdot 10^{6}$", fontsize=34)
#plt.scatter(np.imag(Alm_VarSig*10**3), np.real(Alm_VarSig*10**3), label="VarSig, $l_{\mathsf{max}}=$"+str(l_max), color='g')
plt.scatter(np.imag(Alm_BGD[0]*10**6), np.real(Alm_BGD[0]*10**6), label="BGD without $m=0$, $l_{\mathsf{max}}=$"+str(l_max), color='r')
plt.scatter(np.imag(Alm_BGD_no_m0[0]*10**6), np.real(Alm_BGD_no_m0[0]*10**6), label="BGD without $m=0$, $l_{\mathsf{max}}=$"+str(l_max), color='g')

plt.legend()
plt.gcf().set_size_inches(15, 11)
plt.savefig(SAVE_DIRECTORY+"alm_vs_blm_test.png", dpi=300)
plt.show()


In [42]:
# Plotting alm vs blm:
plt.xlabel("Re($a^{l}_{m}$)$\cdot 10^{6}$", fontsize=34)
plt.ylabel("Re($b^{l}_{m}$)$\cdot 10^{6}$", fontsize=34)

plt.scatter(np.real(Alm_cat*10**6), np.real(Alm_VarSig6*10**6), label="VarSig, $l_{\mathsf{max}}=$"+str(l_max), color='r')
plt.scatter(np.real(Alm_cat*10**6), np.real(Alm_VarSig4*10**6), label="VarSig, $l_{\mathsf{max}}=$"+str(l_max), color='b')
plt.scatter(np.real(Alm_cat*10**6), np.real(Alm_VarSig4_noBG*10**6), label="VarSig, $l_{\mathsf{max}}=$"+str(l_max), color='g')
#plt.scatter(np.real(Alm_cat*10**6), np.real(Alm_cat*10**6), label="Catalogue, $l_{\mathsf{max}}=$"+str(l_max), color='magenta')


plt.legend()
plt.gcf().set_size_inches(15, 11)
#plt.savefig(SAVE_DIRECTORY+"almVsBlmTestImag.png", dpi=300)
plt.show()


In [169]:
#histogram with Gaussian fit:
num_bins = 30
plt.xlabel("Re($a^{l}_{m}$)$\cdot 10^{5}$", fontsize=34)
plt.ylabel("Amplitude [a.u.]", fontsize=34)


x=np.abs(Alm_VarSig3)*100000
mu = np.mean(x) # mean
sigma = np.std(x) # standard deviation 
n, bins, patches = plt.hist(x,num_bins, normed = True, label="BGD, $l_{\mathsf{max}}=$"+str(l_max))
y = mlab.normpdf(bins, mu, sigma)
plt.plot(bins, y, 'r--', label="Gauss fit")
plt.legend()
plt.gcf().set_size_inches(15, 11)
plt.savefig(SAVE_DIRECTORY+"histo_BGD_Re.png", dpi=300)
plt.show()

# Cross-correlation:

In [24]:
selfCat=H.alm2cl(Alm_cat, lmax=1000)
selfBGD=H.alm2cl(Alm_BGD_avg, lmax=1000)
CatXBGD=H.alm2cl(Alm_cat, Alm_BGD_avg, lmax=1000)
CatXVarSig=H.alm2cl(Alm_cat, Alm_VarSig, lmax=1000)

#selfCat2=H.anafast(catalog_map, lmax=1000)
#CatXSth=H.anafast(catalog_map,mergeGPU.map_delta, lmax=1000)
#SthXSth=H.anafast(mergeGPU.map_delta, lmax=1000)

# Plotting $C_l$

In [27]:
plt.xlabel("$l$", fontsize=34)
plt.ylabel("$C_{l}$", fontsize=34)
plt.xlim([0, 1000])
plt.ylim([10**(-9), 10**(0)])
#plt.plot(range(len(crossCl)),abs(np.real(crossCl)), label="crossCl real PureSig x Bgd")
#plt.plot(range(len(crossCl)),abs(np.imag(crossCl)), label="crossCl imag PureSig x Bgd")
#plt.plot(range(len(crossCl)),abscrossCl, label="crossCl abs PureSig x Bgd")
#plt.plot(range(len(crossCl_avg)),abs(np.real(crossCl_avg)), label="crossCl real VarSig x Bgd")
#plt.plot(range(len(crossCl_avg)),abs(np.imag(crossCl_avg)), label="crossCl imag VarSig x Bgd")
#plt.plot(range(len(abscrossCl_avg)),abs(np.imag(abscrossCl_avg)), label="crossCl abs VarSig x Bgd")
#plt.plot(range(len(crossCl)),abs(np.real(crossCl_avgBGD)), label="crossCl real PureSig x BggAavg")
#plt.plot(range(len(crossCl)),abs(np.imag(crossCl_avgBGD)), label="crossCl imag PureSig x BgdAvg")

#plt.plot(range(len(abscrossCl4)),abscrossCl4, label="abs, $\mu=100$", color = 'r')
#plt.plot(range(len(abscrossCl4)),np.abs(crossCl4), label="abs whole, $\mu=100$", color = 'r')
#plt.plot(range(len(crossCl4)),abs(np.real(crossCl4)), label="crossCl real VarSig x Map", color = 'r')
#plt.plot(range(len(crossCl4)),abs(np.imag(crossCl4)), label="crossCl imag VarSig x Map", color = 'r')

#plt.plot(range(len(abscrossCl4b)),abscrossCl4b, label="abs, $\mu=1000$", color = 'g')
#plt.plot(range(len(abscrossCl4b)),np.abs(crossCl4b), label="abs whole, $\mu=1000$", color = 'g')
#plt.plot(range(len(abscrossCl4b)),np.real(crossCl4b), label="real VarSig x Map", color = 'g')
#plt.plot(range(len(abscrossCl4b)),np.imag(crossCl4b), label="imag crossCl b VarSig x Map", color = 'g')

#plt.plot(range(len(abscrossCl4c)),abscrossCl4c, label="abs , $\mu=100000$", color = 'b')
#plt.plot(range(len(abscrossCl4c)),np.abs(crossCl4c), label="abs whole, $\mu=100000$", color = 'b')
#plt.plot(range(len(abscrossCl4c)),np.real(crossCl4c), label="real crossCl c VarSig x Map", color = 'b')
#plt.plot(range(len(abscrossCl4c)),np.imag(crossCl4c), label="imag crossCl c VarSig x Map", color = 'b')

#plt.plot(range(len(abscrossCl4cOff)),abscrossCl4cOff, label="abs , $\mu=100000$ off", color = 'r')
#plt.plot(range(len(abscrossCl4cOff)),np.abs(crossCl4cOff), label="abs whole, $\mu=100000$ off", color = 'r')
#plt.plot(range(len(abscrossCl4cOff)),np.real(crossCl4cOff), label="real crossCl c VarSig x Map", color = 'b')
#plt.plot(range(len(abscrossCl4cOff)),np.imag(crossCl4cOff), label="imag crossCl c VarSig x Map", color = 'b')


#abscrossCl4d = [x /(.046327/.00816449) for x in abscrossCl4d]
#plt.plot(range(len(abscrossCl4d)),abscrossCl4d, label="10 strongest abs, $\mu=10000$", color = 'magenta')
#plt.plot(range(len(abscrossCl4d)),np.abs(crossCl4d), label="abs whole 10 strongest, $\mu=10000$", color = 'magenta')
#plt.plot(range(len(abscrossCl4d)),np.real(crossCl4d), label="real crossCl d VarSig x Map", color = 'magenta')
#plt.plot(range(len(abscrossCl4d)),np.imag(crossCl4d), label="imag crossCl d VarSig x Map", color = 'magenta')

#plt.plot(range(len(abscrossCl4d_noBG)),abscrossCl4d_noBG, label="10 strongest abs no BGD, $\mu=10000$", color = 'orange')
#plt.plot(range(len(abscrossCl4d_noBG)),np.abs(crossCl4d_noBG), label="abs whole 10 strongest no BGD, $\mu=10000$", color = 'orange')
#plt.plot(range(len(abscrossCl4d_noBG)),np.real(crossCl4d_noBG), label="real 10 strongest no BGD", color = 'orange')
#plt.plot(range(len(abscrossCl4d_noBG)),np.imag(crossCl4d_noBG), label="imag 10 strongest no BGD", color = 'orange')

#plt.plot(range(len(abscrossCl4e)),abscrossCl4e, label="10 weakest abs, $\mu=10000$", color = 'cyan')
#plt.plot(range(len(abscrossCl4e)),np.abs(crossCl4e), label="abs whole 10 weakest, $\mu=10000$", color = 'cyan')
#plt.plot(range(len(abscrossCl4e)),np.real(crossCl4e), label="real crossCl e VarSig x Map", color = 'cyan')
#plt.plot(range(len(abscrossCl4e)),np.imag(crossCl4e), label="imag crossCl e VarSig x Map", color = 'cyan')

#plt.plot(range(len(abscrossCl4f)),abscrossCl4f, label="100 strongest abs, $\mu=100$", color = 'brown')
#plt.plot(range(len(abscrossCl4f)),np.abs(crossCl4f), label="abs whole 100 strongest , $\mu=100$", color = 'brown')
#plt.plot(range(len(abscrossCl4f)),np.real(crossCl4f), label="real 100 strongest , $\mu=100$", color = 'cyan')
#plt.plot(range(len(abscrossCl4f)),np.imag(crossCl4f), label="imag 100 strongest , $\mu=100$", color = 'cyan')


#plt.plot(range(len(crossCl6)),abscrossCl6, label="crossCl abs PureSig x Map")
#plt.plot(range(len(selfPure)),selfPure, label="crossCl PureSig x PureSig")
#plt.plot(range(len(selfVarSig)),selfVarSig, label="VarSig x VarSig, $\mu=100$", color = 'black')
#plt.plot(range(len(selfCat)),selfCat, label="Cat x Cat", color = 'black')
#plt.plot(range(len(selfCat2)),selfCat2, label="Cat2 x Cat2", color = 'black')
#plt.plot(range(len(BGDxCat)),BGDxCat, label="BGD x Cat2", color = 'r')
#plt.plot(range(len(VarSigxCat)),VarSigxCat, label="BGD x Cat2", color = 'g')
#plt.scatter(range(len(test)),test, label="Cat x Cat", color = 'black', s=1)
plt.scatter(range(len(selfCat)),selfCat, label="Cat x Cat", color = 'black', s=10)
plt.scatter(range(len(CatXBGD)),CatXBGD, label="Cat x BGD_avg", color = 'red', s=10)
plt.scatter(range(len(selfBGD)),selfBGD, label="BGD_avg x BGD_avg", color = 'blue', s=10)
plt.scatter(range(len(CatXVarSig)),CatXVarSig, label="Cat x VarSig", color = 'green', s=10)

#plt.scatter(range(len(test2)),test2, label="Cat x VarSigExtended, $\mu=100$", color = 'r', s=10)
#plt.scatter(range(len(test3)),test3, label="VarSigExtended x VarSigExtended, $\mu=100$", color = 'b', s=10)
#plt.plot(range(len(selfBGD_avg)),selfBGD_avg, label="BGD x BGD", color = 'black')
#plt.plot(range(len(selfPure)),selfBGD2, label="crossCl BGD x BGD avg", color = 'brown')
#plt.plot(range(len(crossCl7)),abs(np.real(crossCl7)), label="crossCl real PureSigMu15 x Map")
#plt.plot(range(len(crossCl7)),abs(np.imag(crossCl7)), label="crossCl imag PureSigMu15 x Map")
#plt.plot(range(len(crossCl7)),abscrossCl7, label="crossCl abs PureSigMu15 x Map")
plt.yscale('log', basey = 10)
#plt.plot(range(len(crossCl3)),np.log10(np.imag(crossCl3)), label="crossCl3 imag")
#plt.plot(range(len(crossCl3)),np.log10(np.real(crossCl3)), label="crossCl3 real")
#plt.plot(range(l_max+1),np.imag(crossCl2)-np.imag(crossCl), label="crossCl2 imag")
plt.legend()
plt.gcf().set_size_inches(15, 11)
#plt.savefig(SAVE_DIRECTORY+"Cl_CatxBGD_with_plane.png", dpi=300)
plt.show()

In [60]:
# max reasonable l:
test=H.nside2resol(512)
print 180/np.degrees(test)
print 3*512-1

1571.82982341
1535


In [23]:
print min(abs(CatXSth))
print SthXSth[3]
print selfCat[2]

1.28414757711e-12
1.59108269726e-05
6.12877026452e-06


In [29]:
#atmGPU[id].plotLastSkymap(title=SIGNAL+", "+SMEARING_METHOD, nFigure=1, logz=False)
nFigure=7
title=SIGNAL+", "+SMEARING_METHOD

grid=True
logz=True

plotControl(nFigure)
Norm=None
if logz: Norm='log'

smoothed_map = H.smoothing(catalog_map, sigma=np.radians(0.5))
if min(smoothed_map)<=0:
    raiser=abs(min(smoothed_map))*2
else:
    raiser = 0.
    
for i in range(len(smoothed_map)):
    smoothed_map[i] = float(smoothed_map[i])
    if logz:
        smoothed_map[i]=smoothed_map[i]+raiser #does not affect effCl, only overall rise

colormap=plt.get_cmap("Blues")
colormap.set_under("1")
col2="0."
col="0."
res=800

H.mollview(smoothed_map, cmap=colormap, title="", fig=nFigure, xsize=res, norm=Norm) #H.smoothing(, sigma=np.radians(0.5)
H.graticule(dpar=30,dmer=60)


H.projplot(zen2dec(mergeGPU.theta_sou), mergeGPU.phi_sou, color="orange", alpha=0.3, marker="o", ms=50., mew=2., mec="orange", lw=0.)

if grid:
    
    H.projtext(58.0, -8.0, "$60^{\circ}$",lonlat=True, color=col2)
    H.projtext(118.0, -8.0, "$120^{\circ}$",lonlat=True, color=col2)
    H.projtext(178.0, -8.0,"$180^{\circ}$",lonlat=True, color=col2)
    
    H.projtext(-2.0, -8.0, "$0^{\circ}$",lonlat=True, color=col2)
    H.projtext(-62.0, -8.0, "$300^{\circ}$",lonlat=True, color=col2)
    H.projtext(-122.0, -8.0, "$240^{\circ}$",lonlat=True, color=col2)

    H.projtext(-2.0, 22.0, "$30^{\circ}$",lonlat=True, color=col)
    H.projtext(-2.0, 52.0, "$60^{\circ}$",lonlat=True, color=col)
    H.projtext(-2.0, 76.0, "$90^{\circ}$",lonlat=True, color=col)

    H.projtext(-2.0, -28.0, "$-30^{\circ}$",lonlat=True, color=col)
    H.projtext(-2.0, -58.0, "$-60^{\circ}$",lonlat=True, color=col)
    H.projtext(-2.0, -82.0, "$-90^{\circ}$",lonlat=True, color=col)

Sigma is 30.000000 arcmin (0.008727 rad) 
-> fwhm is 70.644601 arcmin
Sigma is 0.000000 arcmin (0.000000 rad) 
-> fwhm is 0.000000 arcmin
0.0 180.0 -180.0 180.0
The interval between parallels is 30 deg -0.00'.
The interval between meridians is 60 deg -0.00'.


In [30]:
plt.show()

# Checking energy & zenith

In [37]:
id=2

In [2]:
plt.figure(3)
#n, bins = np.histogram(np.cos(dec2zen(atmGPU[id].sample_atm["dec"])), bins=80, density=True) #theta values of events in map
#n = normHist(n, 0.95)
#plt.plot(setNewEdges(bins), n, drawstyle="steps-mid", label="cos(sampleTheta) atm")

#.sample_astro["dec"]
n, bins = np.histogram(np.cos(atmGPU[id].ranTheta[:]), bins=80, density=True) #theta values of events in map
n = normHist(n, 0.95)
plt.plot(setNewEdges(bins), n, drawstyle="steps-mid", label="ran theta all", color="green")
plt.legend(loc=2)
plt.show()

TypeError: list indices must be integers, not builtin_function_or_method

In [12]:
plt.figure(4)

n, bins = np.histogram(atmGPU[id].ranPhi[:], bins=80, density=True) #theta values of events in map
n = normHist(n, 0.95)
plt.plot(setNewEdges(bins), n, drawstyle="steps-mid", label="ran phi all", color="red")
plt.legend(loc=2)
plt.show()

In [13]:
nmc, binsmc = np.histogram(np.cos(np.array(dec2zen(atmGPU[id].mc_sample_full["dec"]))), bins=100, weights=atmGPU[id].mc_sample_full["conv"], density=True) #theta values of mc atm
nmc = normHist(nmc, 0.95)
plt.plot(setNewEdges(binsmc), nmc, drawstyle="steps-mid", label="cos(fullTheta) atm")

nmc, binsmc = np.histogram(np.cos(np.array(dec2zen(atmGPU[id].mc_sample_full["dec"]))), bins=100, weights=atmGPU[id].mc_sample_full["astro"], density=True) #theta values of mc atm
nmc = normHist(nmc, 0.95)
plt.plot(setNewEdges(binsmc), nmc, drawstyle="steps-mid", label="cos(fullTheta) astro")

plt.legend(loc="lower left", fontsize = 'x-small')
plt.show()

In [49]:
print len(atmGPU[id].sample_astro["dec"])

258


In [50]:
#plt.close("all")

In [51]:
saveAll([1, 3, 4])

saved figure (1) successfully.
saved figure (3) successfully.
saved figure (4) successfully.


True

In [38]:
def plotEnergyDistr(nFigure=-1, nLabel=['Diff. Background', 'Atm. Background'], normed=False):
    plotControl(nFigure)
    nColor=["lightblue", "yellow"]

    cut=len(atmGPU[id].diffElist)
    purebg=atmGPU[id].energyList[cut:]

    plt.hist([atmGPU[id].diffElist, purebg], bins= 40, range=(hmin,hmax), stacked=True, color=nColor, label=nLabel, normed=normed)
    plt.semilogy(nonposy="clip")
    plt.legend()
    plt.title("Event Energy Spectrum")
    plt.xlabel(r"log(E)[GeV]")
    plt.ylabel("#")
    
hmin=min(atmGPU[id].energyList)
hmax=max(atmGPU[id].energyList)
print "min energy: "+str(hmin)
print "max energy: "+str(hmax)    
plotEnergyDistr(2, normed=True)
plt.plot(atmGPU[id].atmE_Edges, atmGPU[id].atmEnergySpline(atmGPU[id].atmE_Edges)+atmGPU[id].sigEnergySpline(atmGPU[id].sigE_Edges)/100.)
plt.plot(atmGPU[id].sigE_Edges, atmGPU[id].sigEnergySpline(atmGPU[id].sigE_Edges)/100.)
plt.ylim([1e-5, 1])
plt.xlim([hmin, hmax])
plt.show()

IndexError: list index out of range

# Skymap

In [27]:
id=2
atmGPU[id].plotLastSkymap(title=SIGNAL+", "+SMEARING_METHOD, nFigure=5, logz=True)
plt.show()

0.0 180.0 -180.0 180.0
The interval between parallels is 30 deg -0.00'.
The interval between meridians is 60 deg -0.00'.


# Rotation check

In [28]:
from matplotlib.colors import LogNorm
# test plots
plt.figure(13)
f = plt.scatter(np.degrees(atmGPU[id].ps_sample["ra"]), np.degrees(atmGPU[id].ps_sample["dec"]))
#plt.xlim(0, 360)

#plt.ylim(-10,90)
plt.xlabel("RA/deg")
plt.ylabel("Dec/deg")

plt.figure(14)
color = ["blue", "lightgreen", "orange", "red", "magenta", "darkblue", "green", "brown", "darkred", "purple"]
i=0
#colors = ["r","g","b", "c", "orange", "k"]
for rotDec in list(set(atmGPU[id].ps_sample["trueRotDec"])):
    #print i
    plt.scatter(np.degrees(atmGPU[id].ps_sample["trueRa"][np.where(atmGPU[id].ps_sample["trueRotDec"] == rotDec)]), 
                np.degrees(atmGPU[id].ps_sample["trueDec"][np.where(atmGPU[id].ps_sample["trueRotDec"] == rotDec)]),
                color=color[i%10])
    i+=1
plt.xlim(0, 360)
plt.show()